In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# change the path to where your notebook is located
%cd "gdrive/My Drive/Colab Notebooks"

/content/gdrive/My Drive/Colab Notebooks


In [ ]:
!pip install datasets
!pip install transformers
!pip install transformers[torch]
!pip install accelerate -U
import numpy as np
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.6 MB/s eta 0:00:00


In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [ ]:
# Run it only once
from datasets import load_dataset
dataset = load_dataset("multi_woz_v22")

Extracting data files:   0%|          | 0/22 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/8437 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Run it only once
dataset.save_to_disk("dataset.hf")

Saving the dataset (0/1 shards):   0%|          | 0/8437 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from datasets import DatasetDict
dataset = DatasetDict.load_from_disk("dataset.hf")

In [ ]:
dataset['train'][0]['turns']

In [ ]:
def make_df(split):

  train_domain = []
  conjunto = set()
  extracted_information_ground_truth=[]

  for d,dial in enumerate(dataset[split]):
      # print(dial)
      # print(dial['dialogue_id'])
      speaker_str = {0: 'User', 1: 'Agent'}
      turns = dial['turns']
      if not any(set(dial['turns']['frames'][turn_id]['service']).intersection(['hotel', 'restaurant']) for turn_id,utt in enumerate(dial['turns']['utterance'])):
        continue
      for turn_id,utt in enumerate(turns['utterance']):
          extracted_information_ground_truth = []
          start = []
          end = []
          already = set()
          speaker = speaker_str[turns['speaker'][turn_id]]
          # if speaker == "User":
          dialogue_acts = turns['dialogue_acts'][turn_id]['dialog_act']['act_type']
          acts_to_add = []
          for act in dialogue_acts:
            if 'Attraction' in act or 'Train' in act or 'Hospital' in act or 'Taxi' in act or 'Police' in act:
              act_single = act.split('-')
              if act_single[0] not in already:
                acts_to_add.append(act_single[0])
                conjunto.add(act_single[0])
                already.add(act_single[0])
            else:

              acts_to_add.append(act)
              conjunto.add(act)
          #if(len(acts_to_add)) > 0:
            #train_domain.append([dial['dialogue_id'], utt, turn_id, speaker, acts_to_add])
          dialogue_acts_slots = turns['dialogue_acts'][turn_id]['span_info']['act_type']
          slot_names_per_act = turns['dialogue_acts'][turn_id]['span_info']['act_slot_name']
          slot_values_per_act = turns['dialogue_acts'][turn_id]['span_info']['act_slot_value']
          span_values_start = turns['dialogue_acts'][turn_id]['span_info']['span_start']
          span_values_end = turns['dialogue_acts'][turn_id]['span_info']['span_end']

          for act_i in range(len(slot_names_per_act)):

              if dialogue_acts_slots[act_i].startswith("Hotel") or dialogue_acts_slots[act_i].startswith("Restaurant"):
                if len(slot_names_per_act) > 0:
                  span_start = span_values_start[act_i]
                  span_end = span_values_end[act_i]
                  extracted_information_ground_truth.append(tuple([dialogue_acts_slots[act_i].split("-")[0].lower()+"-"+slot_names_per_act[act_i], slot_values_per_act[act_i]]))
                  start.append(span_start)
                  end.append(span_end)

          if (len(extracted_information_ground_truth) > 0 or (len(acts_to_add)) > 0):
            train_domain.append([dial['dialogue_id'], utt, turn_id, speaker, acts_to_add, extracted_information_ground_truth, start, end])

  frame=pd.DataFrame(train_domain, columns=['conv_id', 'utterance', 'turn_id', 'speaker', 'acts','slots_dict','slots_begin','slots_end'])
  columns_to_remove=['conv_id','turn_id']
  #return frame
  frame.drop(columns=columns_to_remove,inplace=True)
  return frame



In [ ]:
train_df = make_df('train')
val_df = make_df('validation')
test_df = make_df('test')

In [ ]:
train_df

,utterance,speaker,acts,slots_dict,slots_begin,slots_end
0,i need a place to dine in the center thats exp...,User,[Restaurant-Inform],"[(restaurant-area, centre), (restaurant-pricer...","[30, 43]","[36, 52]"
1,I have several options for you; do you prefer ...,Agent,"[Restaurant-Inform, Restaurant-Select]","[(restaurant-choice, several), (restaurant-foo...","[7, 46, 55, 65]","[14, 53, 60, 72]"
2,"Any sort of food would be fine, as long as it ...",User,[Restaurant-Request],[],[],[]
3,There is an Afrian place named Bedouin in the ...,Agent,[Restaurant-Inform],"[(restaurant-food, Afrian), (restaurant-name, ...","[12, 31, 46]","[18, 38, 52]"
4,"Sounds good, could I get that phone number? Al...",User,"[Hotel-Inform, Restaurant-Request]","[(hotel-pricerange, expensive), (hotel-type, h...","[76, 86]","[85, 91]"
...,...,...,...,...,...,...
90646,nandos serves portuguese food and in the cheap...,Agent,"[Restaurant-Recommend, general-reqmore]","[(restaurant-name, nandos), (restaurant-food, ...","[0, 14, 41]","[6, 24, 46]"
90647,"I would like the address of Nandos restaurant,...",User,"[Restaurant-Inform, Restaurant-Request]","[(restaurant-name, nandos)]",[28],[34]
90648,Nandos is located in the south part of the cit...,Agent,"[Restaurant-Inform, general-reqmore]","[(restaurant-name, Nandos), (restaurant-area, ...","[0, 25, 67]","[6, 47, 101]"
90649,"Thank you, goodbye.",User,[general-bye],[],[],[]


In [ ]:
test_df

,utterance,speaker,acts,slots_dict,slots_begin,slots_end
0,"Hello, I am looking for a restaurant in Cambri...",User,[Restaurant-Inform],"[(restaurant-name, golden wok)]",[74],[84]
1,It is located at 191 Histon Road Chesterton,Agent,[Restaurant-Inform],"[(restaurant-address, 191 Histon Road Chestert...",[17],[43]
2,Can you book me a table for 11:00 on Friday?,User,[Restaurant-Inform],"[(restaurant-booktime, 11:00), (restaurant-boo...","[28, 37]","[33, 43]"
3,Yes I can! Table for 1?,Agent,[Booking-Inform],[],[],[]
4,"Actually, for 4, please.",User,[Restaurant-Inform],"[(restaurant-bookpeople, 4)]",[14],[15]
...,...,...,...,...,...,...
11464,It looks as though your train has already been...,Agent,[Train],[],[],[]
11465,"Thank you, can you please tell me the travel t...",User,[Train],[],[],[]
11466,The price is indeed 13.20 Pounds and the durat...,Agent,"[Train, general-reqmore]",[],[],[]
11467,"No, that would be all. Thank you.",User,[general-thank],[],[],[]


In [ ]:
train_df['acts_no_list'] = train_df['acts'].apply(lambda x: ', '.join(map(str, x)))

In [ ]:
train_df

,utterance,speaker,acts,slots_dict,slots_begin,slots_end,acts_no_list,slots_no_list
0,i need a place to dine in the center thats exp...,User,[Restaurant-Inform],"[(restaurant-area, centre), (restaurant-pricer...","[30, 43]","[36, 52]",Restaurant-Inform,"('restaurant-area', 'centre'), ('restaurant-pr..."
1,I have several options for you; do you prefer ...,Agent,"[Restaurant-Inform, Restaurant-Select]","[(restaurant-choice, several), (restaurant-foo...","[7, 46, 55, 65]","[14, 53, 60, 72]","Restaurant-Inform, Restaurant-Select","('restaurant-choice', 'several'), ('restaurant..."
2,"Any sort of food would be fine, as long as it ...",User,[Restaurant-Request],[],[],[],Restaurant-Request,
3,There is an Afrian place named Bedouin in the ...,Agent,[Restaurant-Inform],"[(restaurant-food, Afrian), (restaurant-name, ...","[12, 31, 46]","[18, 38, 52]",Restaurant-Inform,"('restaurant-food', 'Afrian'), ('restaurant-na..."
4,"Sounds good, could I get that phone number? Al...",User,"[Hotel-Inform, Restaurant-Request]","[(hotel-pricerange, expensive), (hotel-type, h...","[76, 86]","[85, 91]","Hotel-Inform, Restaurant-Request","('hotel-pricerange', 'expensive'), ('hotel-typ..."
...,...,...,...,...,...,...,...,...
90646,nandos serves portuguese food and in the cheap...,Agent,"[Restaurant-Recommend, general-reqmore]","[(restaurant-name, nandos), (restaurant-food, ...","[0, 14, 41]","[6, 24, 46]","Restaurant-Recommend, general-reqmore","('restaurant-name', 'nandos'), ('restaurant-fo..."
90647,"I would like the address of Nandos restaurant,...",User,"[Restaurant-Inform, Restaurant-Request]","[(restaurant-name, nandos)]",[28],[34],"Restaurant-Inform, Restaurant-Request","('restaurant-name', 'nandos')"
90648,Nandos is located in the south part of the cit...,Agent,"[Restaurant-Inform, general-reqmore]","[(restaurant-name, Nandos), (restaurant-area, ...","[0, 25, 67]","[6, 47, 101]","Restaurant-Inform, general-reqmore","('restaurant-name', 'Nandos'), ('restaurant-ar..."
90649,"Thank you, goodbye.",User,[general-bye],[],[],[],general-bye,


In [ ]:
def create_agent_context_dataframe(df):
  context_user=[]
  agent_acts=[]
  df['acts_no_list'] = df['acts'].apply(lambda x: ', '.join(map(str, x)))
  df['slots_no_list'] = df['slots_dict'].apply(lambda x: ', '.join(map(str, x)))
  for index, utt in enumerate(df['utterance'].tolist()):
    if df['speaker'][index]=='Agent' and df['speaker'][index-1]=='User':
      context_user_utterance = df['utterance'][index-1]
      context_user_acts = df['acts_no_list'][index-1]
      context_user_slots_dict = df['slots_no_list'][index-1]
      context_user.append(f"{context_user_utterance} | {context_user_acts} | {context_user_slots_dict}")
      agent_acts.append(df['acts'][index])

  result_df = pd.DataFrame({'context_user': context_user, 'agent_acts': agent_acts})

  return result_df



In [ ]:
train_df_comp = create_agent_context_dataframe(train_df)
val_df_comp = create_agent_context_dataframe(val_df)
test_df_comp = create_agent_context_dataframe(test_df)

In [ ]:
test_df_comp.to_csv('agent_dialog_acts.csv', index=False)

In [ ]:
pd.read_csv('agent_dialog_acts.csv')

,context_user,agent_acts
0,"Hello, I am looking for a restaurant in Cambri...",['Restaurant-Inform']
1,Can you book me a table for 11:00 on Friday? |...,['Booking-Inform']
2,"Actually, for 4, please. | Restaurant-Inform |...",['Booking-Book']
3,"Great, can you also get me information or arch...",['Attraction']
4,Can I get the postcode for that? I also need t...,"['Attraction', 'Taxi']"
...,...,...
5676,What area of town is the hotel in? | Hotel-Req...,"['Booking-Inform', 'Hotel-Inform']"
5677,Does it have free parking? | Hotel-Request |,"['Booking-Inform', 'Hotel-Inform', 'general-re..."
5678,That's alright. I can book it later. I'm also ...,['Train']
5679,"Thank you, can you please tell me the travel t...","['Train', 'general-reqmore']"


In [ ]:
train_df_comp = create_agent_context_dataframe(train_df)
val_df_comp = create_agent_context_dataframe(val_df)
test_df_comp = create_agent_context_dataframe(test_df)

In [ ]:
train_df_comp['context_user'][1]

"I have several options for you; do you prefer African, Asian, or British food? | Restaurant-Inform, Restaurant-Select | ('restaurant-choice', 'several'), ('restaurant-food', 'African'), ('restaurant-food', 'Asian'), ('restaurant-food', 'British')"

In [ ]:
def encoding(df):
  dummies = pd.get_dummies(df['agent_acts'].apply(pd.Series).stack()).sum(level=0)

  # Concatenate the dummies DataFrame with the original DataFrame
  result = pd.concat([df, dummies], axis=1)

  # Drop the original 'acts' column
  result = result.drop('agent_acts', axis=1)
  return result

# Fill NaN values with 0
# result = result.fillna(0).astype(int)

In [ ]:
train_df_enc = encoding(train_df_comp)
val_df_enc = encoding(val_df_comp)
test_df_enc = encoding(test_df_comp)

<ipython-input-14-3b7f4a0221fb>:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(df['agent_acts'].apply(pd.Series).stack()).sum(level=0)
<ipython-input-14-3b7f4a0221fb>:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(df['agent_acts'].apply(pd.Series).stack()).sum(level=0)
<ipython-input-14-3b7f4a0221fb>:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(df['agent_acts'].apply(pd.Series).stack()).sum(level=0)


In [ ]:
labels = [label for label in train_df_enc.columns if label not in ['context_user']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['Attraction',
 'Booking-Book',
 'Booking-Inform',
 'Booking-NoBook',
 'Booking-Request',
 'Hospital',
 'Hotel-Inform',
 'Hotel-NoOffer',
 'Hotel-Recommend',
 'Hotel-Request',
 'Hotel-Select',
 'Police',
 'Restaurant-Inform',
 'Restaurant-NoOffer',
 'Restaurant-Recommend',
 'Restaurant-Request',
 'Restaurant-Select',
 'Taxi',
 'Train',
 'general-bye',
 'general-greet',
 'general-reqmore',
 'general-thank',
 'general-welcome']

In [ ]:
from transformers import AutoTokenizer
# import numpy as np

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def preprocessing(examples):
    # take a batch of texts
    text = examples["context_user"]
    # encode them
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)

    # add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}

    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(text), len(labels)))

    # fill numpy array
    for idx, label in enumerate(labels):
        # Check if the label is present in the batch
        if label in labels_batch:
            labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()

    return encoding


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from datasets import Dataset


In [ ]:
train_dataset = Dataset.from_pandas(train_df_enc)
test_dataset = Dataset.from_pandas(test_df_enc)
val_dataset = Dataset.from_pandas(val_df_enc)

In [ ]:
encoded_train = train_dataset.map(preprocessing, batched=True, remove_columns=train_dataset.column_names)
encoded_test = test_dataset.map(preprocessing, batched=True, remove_columns=test_dataset.column_names)
encoded_val = val_dataset.map(preprocessing, batched=True, remove_columns=val_dataset.column_names)

Map:   0%|          | 0/44862 [00:00<?, ? examples/s]

Map:   0%|          | 0/5681 [00:00<?, ? examples/s]

Map:   0%|          | 0/5814 [00:00<?, ? examples/s]

In [ ]:
encoded_train.set_format("torch")
encoded_test.set_format("torch")
encoded_val.set_format("torch")

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
device = 'cuda'

model = AutoModelForSequenceClassification.from_pretrained("roberta-base",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
batch_size = 16
metric_name = "f1"
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    "confs",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    print(f'y_true = {y_true}')
    print(f'y_pred = {y_pred}')
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def calculate_performance_metrics(predictions: EvalPrediction):
    model_predictions = predictions.predictions[0] if isinstance(predictions.predictions, tuple) else predictions.predictions
    metrics_result = evaluate_multi_label_metrics(
        model_predictions=model_predictions,
        true_labels=predictions.label_ids)
    return metrics_result

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train,
    eval_dataset=encoded_val,
    tokenizer=tokenizer,
    compute_metrics=calculate_performance_metrics
)

In [ ]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.101100,0.089988,0.658457,0.774978,0.435157
2,0.092900,0.085087,0.679024,0.786647,0.452356
3,0.088600,0.084034,0.695607,0.804347,0.469900


y_true = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
y_pred = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
y_true = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
y_pred = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
y_true = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
y_pred = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.101100,0.089988,0.658457,0.774978,0.435157
2,0.092900,0.085087,0.679024,0.786647,0.452356
3,0.088600,0.084034,0.695607,0.804347,0.469900
4,0.086900,0.083187,0.698042,0.808401,0.468696


y_true = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
y_pred = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


TrainOutput(global_step=11216, training_loss=0.09818187691855873, metrics={'train_runtime': 4141.2535, 'train_samples_per_second': 43.332, 'train_steps_per_second': 2.708, 'total_flos': 1.1806019737731072e+16, 'train_loss': 0.09818187691855873, 'epoch': 4.0})

In [ ]:
model.save_pretrained('agent_dialog_acts')

In [ ]:
from transformers import AutoModelForSequenceClassification

# Replace 'path_to_your_model_directory' with the actual path to your saved model directory
model_directory = '/content/gdrive/MyDrive/NLI project/agent_dialog_acts'

# Load the pre-trained BERT model
loaded_model = AutoModelForSequenceClassification.from_pretrained(model_directory)

In [ ]:
import torch

In [ ]:
utt=test_df_comp['context_user'][0]

In [ ]:
utt

"Hello, I am looking for a restaurant in Cambridge. I believe it is called Golden Wok. | Restaurant-Inform | ('restaurant-name', 'golden wok')"

In [ ]:
test_df_comp['agent_acts'][0]

['Restaurant-Inform']

In [ ]:
def predict(utt):
  inputs = tokenizer(utt, return_tensors="pt")
  #inputs.to('cuda')

  with torch.no_grad():
      logits = model(**inputs).logits

  predictions = torch.argmax(logits, dim=1)
  #print(predictions)
  predicted_class=id2label[predictions.item()]
  return predicted_class

In [ ]:
numpy_matrix = train_df_comp.to_numpy(copy=True)

In [ ]:
true_vals = numpy_matrix[:, 4]
true_vals

In [ ]:
pred_vals = [predict(row[1], model) for row in test]

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=list(labels))
y_true_list = [mlb.fit_transform([true_labels]) for true_labels in true_vals]
y_pred_list = [mlb.transform([predicted_labels]) for predicted_labels in pred_vals]


In [ ]:
y_true_list

y_true = []
y_pred = []

for i in range(len(y_true_list)):
  print()
  y_true.append(y_true_list[i][0])
  y_pred.append(y_pred_list[i][0])
print(classification_report(y_true, y_pred,target_names=list(labels)))